Данные взяты отсюда: https://www.kaggle.com/datasets/microize/newyork-yellow-taxi-trip-data-2020-2019/data

In [36]:
import pyspark
from pyspark.sql import SparkSession, DataFrame, types
from pyspark.sql import functions as F

from functools import reduce

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/11/11 19:04:37 WARN Utils: Your hostname, MacBook-Pro-Eva.local resolves to a loopback address: 127.0.0.1; using 10.1.5.116 instead (on interface en0)
24/11/11 19:04:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/11 19:04:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [64]:
df = spark.read \
    .option("header", "true") \
    .csv("data/csv/yellow_tripdata_2020-01.csv")

In [65]:
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2020-01-01 00:28:15|  2020-01-01 00:33:03|              1|         1.20|         1|                 N|         238|         239|           1|          6|    3|    0.5|      1.47|           0|                  0.3

In [66]:
df.schema

StructType([StructField('VendorID', StringType(), True), StructField('tpep_pickup_datetime', StringType(), True), StructField('tpep_dropoff_datetime', StringType(), True), StructField('passenger_count', StringType(), True), StructField('trip_distance', StringType(), True), StructField('RatecodeID', StringType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('payment_type', StringType(), True), StructField('fare_amount', StringType(), True), StructField('extra', StringType(), True), StructField('mta_tax', StringType(), True), StructField('tip_amount', StringType(), True), StructField('tolls_amount', StringType(), True), StructField('improvement_surcharge', StringType(), True), StructField('total_amount', StringType(), True), StructField('congestion_surcharge', StringType(), True)])

In [67]:
schema = types.StructType([
    types.StructField('VendorID', types.IntegerType(), True), 
    types.StructField('tpep_pickup_datetime', types.TimestampType(), True), 
    types.StructField('tpep_dropoff_datetime', types.TimestampType(), True), 
    types.StructField('passenger_count', types.IntegerType(), True), 
    types.StructField('trip_distance', types.DoubleType(), True), 
    types.StructField('RatecodeID', types.IntegerType(), True), 
    types.StructField('store_and_fwd_flag', types.DoubleType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('payment_type', types.IntegerType(), True), 
    types.StructField('fare_amount', types.IntegerType(), True), 
    types.StructField('extra', types.DoubleType(), True), 
    types.StructField('mta_tax', types.DoubleType(), True), 
    types.StructField('tip_amount', types.DoubleType(), True), 
    types.StructField('tolls_amount', types.IntegerType(), True), 
    types.StructField('improvement_surcharge', types.DoubleType(), True), 
    types.StructField('total_amount', types.IntegerType(), True), 
    types.StructField('congestion_surcharge', types.IntegerType(), True)
])

Всего у меня 6 датасетов по месяцам 2020 года  
Я хочу прочитать их все, а потом разбить по партициям

In [68]:
dfs = []

for month in range(1, 7):
    df = spark.read \
        .option("header", "true") \
        .schema(schema) \
        .csv(f"data/csv/yellow_tripdata_2020-0{month}.csv")

    dfs.append(df)

In [69]:
assert len(dfs) == 6
dfs[1].show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2020-02-01 00:17:35|  2020-02-01 00:30:32|              1|          2.6|         1|              NULL|         145|           7|           1|         11|  0.5|    0.5|      2.45|           0|                  0.3

In [70]:
dfs[0].schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', IntegerType(), True), StructField('store_and_fwd_flag', DoubleType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', IntegerType(), True), StructField('fare_amount', IntegerType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', IntegerType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', IntegerType(), True), StructField('congestion_surcharge', IntegerType(), True)])

Проверим все ли данные соотвествуют заявленным годам и месяцам

In [71]:
dfs[0].filter(dfs[0].tpep_pickup_datetime >= "2020-01-31 23:59:59").show(1)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       2| 2021-01-02 00:22:00|  2021-01-02 00:36:50|              1|         1.56|         1|              NULL|         142|         161|           2|       NULL|  1.0|    0.5|       0.0|           0|                  0.3

Есть данные выбивающиеся из диапазона отфильтруем их

In [72]:
days = [31, 29, 31, 30, 31, 30] # кол-во дней в месяцах

for month in range(1, 7):
    i = month - 1
    
    start_date = f"2020-0{month}-01 00:00:00"
    end_date = f"2020-0{month}-{days[i]} 23:59:59"
    
    dfs[i] = dfs[i].filter(
        (dfs[i].tpep_pickup_datetime >= start_date) & 
        (dfs[i].tpep_dropoff_datetime <= end_date)
    )

Выведем минимальную и максимальную дату для каждого датасета для проверки корректности данных

In [73]:
for month in range(1, 7):
    i = month - 1

    min_date = dfs[i].agg({"tpep_pickup_datetime": "min"}).collect()[0]
    max_date = dfs[i].agg({"tpep_pickup_datetime": "max"}).collect()[0]
    
    print(f"Month {month}")
    print(f"Min date: {min_date}")
    print(f"Max date: {max_date}")
    print()

Month 1
Min date: Row(min(tpep_pickup_datetime)=datetime.datetime(2020, 1, 1, 0, 0))
Max date: Row(max(tpep_pickup_datetime)=datetime.datetime(2020, 1, 31, 23, 59, 7))



Month 2
Min date: Row(min(tpep_pickup_datetime)=datetime.datetime(2020, 2, 1, 0, 0))
Max date: Row(max(tpep_pickup_datetime)=datetime.datetime(2020, 2, 29, 23, 58, 40))



Month 3
Min date: Row(min(tpep_pickup_datetime)=datetime.datetime(2020, 3, 1, 0, 0))
Max date: Row(max(tpep_pickup_datetime)=datetime.datetime(2020, 3, 31, 23, 58, 13))

Month 4
Min date: Row(min(tpep_pickup_datetime)=datetime.datetime(2020, 4, 1, 0, 0))
Max date: Row(max(tpep_pickup_datetime)=datetime.datetime(2020, 4, 30, 23, 56, 49))

Month 5
Min date: Row(min(tpep_pickup_datetime)=datetime.datetime(2020, 5, 1, 0, 0))
Max date: Row(max(tpep_pickup_datetime)=datetime.datetime(2020, 5, 31, 23, 54, 26))

Month 6
Min date: Row(min(tpep_pickup_datetime)=datetime.datetime(2020, 6, 1, 0, 0))
Max date: Row(max(tpep_pickup_datetime)=datetime.datetime(2020, 6, 30, 23, 54))



## Преобразования

- [x] Введем колонку `trip_duration_sec = tpep_dropoff_datetime - tpep_pickup_datetime` в секундах
- [x] Отсортируем каждый датафрейм по `tpep_pickup_datetime` asc, а затем по `trip_duration_sec` desc
- [x] Сгруппируем по всем датасетам по `PULocationID` и `tpep_pickup_datetime` и посчитаем сумму `Passenger_count`, среднее `Trip_distance`, среднее `Tip_amount`. Затем сохраним, как партицию
- [x] Такую же группировку для `DOLocationID`, `tpep_dropoff_datetime`
- [x] Сджоиним датасеты с датасетом `taxi+_zone_lookup.csv`

### Колонка trip_duration

In [74]:
for month in range(1, 7):
    i = month - 1

    dfs[i] = dfs[i] \
    .withColumn("trip_duration_sec", dfs[i].tpep_dropoff_datetime.cast("long") - dfs[i].tpep_pickup_datetime.cast("long"))

In [75]:
dfs[0] \
    .select(['tpep_dropoff_datetime', 'tpep_pickup_datetime', 'trip_duration_sec']) \
    .show(5)

+---------------------+--------------------+-----------------+
|tpep_dropoff_datetime|tpep_pickup_datetime|trip_duration_sec|
+---------------------+--------------------+-----------------+
|  2020-01-01 00:33:03| 2020-01-01 00:28:15|              288|
|  2020-01-01 00:43:04| 2020-01-01 00:35:39|              445|
|  2020-01-01 00:53:52| 2020-01-01 00:47:41|              371|
|  2020-01-01 01:00:14| 2020-01-01 00:55:23|              291|
|  2020-01-01 00:04:16| 2020-01-01 00:01:58|              138|
+---------------------+--------------------+-----------------+
only showing top 5 rows



### Сортировка

In [76]:
for month in range(1, 7):
    i = month - 1

    dfs[i] = dfs[i] \
        .orderBy(dfs[i].tpep_pickup_datetime.asc(), dfs[i].trip_duration_sec.desc())

### Группировка

In [78]:
df_all = reduce(DataFrame.unionAll, dfs)

In [79]:
assert df_all.count() == sum(dfs[i].count() for i in range(6))

In [80]:
df_all.createOrReplaceTempView('data')

In [82]:
df_all.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'trip_duration_sec']

In [84]:
df_pick_up_info = spark.sql("""
SELECT 
    PULocationID,
    DATE_TRUNC('day', tpep_pickup_datetime) as pickup_date,
    SUM(Passenger_count) as passenger_sum,
    AVG(Trip_distance) as avg_trip_distance,
    AVG(Tip_amount) as avg_tip_amount

FROM
    data

GROUP BY
    PULocationID, DATE_TRUNC('day', tpep_pickup_datetime)
""")

In [85]:
df_drop_off_info = spark.sql("""
SELECT 
    DOLocationID,
    DATE_TRUNC('day', tpep_dropoff_datetime) as dropoff_date,
    SUM(Passenger_count) as passenger_sum,
    AVG(Trip_distance) as avg_trip_distance,
    AVG(Tip_amount) as avg_tip_amount

FROM
    data

GROUP BY
    DOLocationID, DATE_TRUNC('day', tpep_dropoff_datetime)
""")

In [86]:
df_pick_up_info \
    .repartition(20) \
    .write.parquet("data/parquet/prepared/pickup_2020")

24/11/11 20:35:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:35:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:35:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:35:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:35:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:35:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:35:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:35:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:35:57 WARN RowBasedKeyValueBatch: Calling spill() on

In [87]:
df_drop_off_info \
    .repartition(20) \
    .write.parquet("data/parquet/prepared/dropoff_2020")

24/11/11 20:36:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:36:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:36:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:36:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:36:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:36:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:36:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:36:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/11 20:36:47 WARN RowBasedKeyValueBatch: Calling spill() on

### Слияние

In [94]:
taxi_zone_df = spark.read \
    .option("header", "true") \
    .csv("data/csv/taxi+_zone_lookup.csv", sep=";")

In [95]:
taxi_zone_df.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [98]:
for month in range(1, 7):
    i = month - 1
    
    dfs[i] = dfs[i] \
        .join(taxi_zone_df, dfs[i].PULocationID == taxi_zone_df.LocationID, how='left') \
        .drop("LocationID")

## Выгрузка данных

Теперь разобьем датасет на партиции

In [99]:
for month in range(1, 7):
    i = month - 1
    
    output_path = f"data/parquet/2020/0{month}"
    
    dfs[i] \
        .repartition(10) \
        .write.parquet(output_path, mode="overwrite")

24/11/11 20:51:12 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/11/11 20:51:18 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/11/11 20:51:19 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/11/11 20:52:40 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/11/11 20:52:48 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/11/11 20:52:48 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/11/11 20:53:30 WARN MemoryManager: Total allocation exceeds 95,00% 

In [100]:
!ls data/parquet/2020

01 02 03 04 05 06


In [101]:
!ls data/parquet/2020/01

_SUCCESS
part-00000-1ac59635-79e5-4ec2-aef5-22b52f848a02-c000.snappy.parquet
part-00001-1ac59635-79e5-4ec2-aef5-22b52f848a02-c000.snappy.parquet
part-00002-1ac59635-79e5-4ec2-aef5-22b52f848a02-c000.snappy.parquet
part-00003-1ac59635-79e5-4ec2-aef5-22b52f848a02-c000.snappy.parquet
part-00004-1ac59635-79e5-4ec2-aef5-22b52f848a02-c000.snappy.parquet
part-00005-1ac59635-79e5-4ec2-aef5-22b52f848a02-c000.snappy.parquet
part-00006-1ac59635-79e5-4ec2-aef5-22b52f848a02-c000.snappy.parquet
part-00007-1ac59635-79e5-4ec2-aef5-22b52f848a02-c000.snappy.parquet
part-00008-1ac59635-79e5-4ec2-aef5-22b52f848a02-c000.snappy.parquet
part-00009-1ac59635-79e5-4ec2-aef5-22b52f848a02-c000.snappy.parquet
